In [58]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser=Browser("chrome", **executable_path, headless=False)

In [3]:
# create function soup for the usniversal usage for several urls
def sooup(url):
    browser.visit(url)
    html=browser.html
    soup=bs(html,'lxml')
    return soup

In [4]:
# all the urls we will use
url1="https://en.wikipedia.org/wiki/List_of_covers_of_Time_magazine_(1970s)"
url2="https://en.wikipedia.org/wiki/List_of_covers_of_Time_magazine_(1980s)"
url3="https://en.wikipedia.org/wiki/List_of_covers_of_Time_magazine_(1990s)"
url4="https://en.wikipedia.org/wiki/List_of_covers_of_Time_magazine_(2000s)"
url5="https://en.wikipedia.org/wiki/List_of_covers_of_Time_magazine_(2010s)"

### 1970s, 1980s data scrapings: text type

In [5]:
# create a function 'titleA' for getting the names in 1970s and 1980s
def titleA(sooup):
    names_by_year={}
    names = []
    spans = sooup.find_all('span', class_='mw-headline')
    years=[y.text for y in spans]

    for idx, yearr in enumerate(years):
        names_by_year[yearr] = []
        current_span = spans[idx]
        header = current_span.parent
        div = header.find_next('div')
        
        lis = div.find_all('li')
        for l in lis:
            try:
                cover_name=l.find('a').attrs['title']
                names_by_year[yearr].append(cover_name)
                names.append({
                    'name': cover_name,
                    'year': yearr
                })
#                 print(cover_name)

            except:
                pass
        
    return names

In [8]:
# 1970s(1977 starts from 269)
names1=titleA(sooup(url1)) #1970s
df1=pd.DataFrame(names1)
#df1=df1[269:385]
df1.head()
#df1.count() 386 total

,name,year
0,Middle America (United States),1970
1,The Band,1970
2,Najeeb Halaby,1970
3,Biafra,1970
4,Barry Commoner,1970


In [9]:
names2=titleA(sooup(url2)) #1980s
names2
df2=pd.DataFrame(names2)
df2.head()
#df2.count() #283
#df2['name'].unique()
#df2['name'].unique_value()

,name,year
0,Ruhollah Khomeini,1980
1,Jimmy Carter,1980
2,Eric Heiden,1980
3,Dan Rather,1980
4,Peter Sellers,1980


### 1990s, 2000s, 2010s ata scrapings: table type

In [10]:
# create a data with name/year
def titleB(sooup):
    names_by_year={}
    names = []
    spans = sooup.find_all('span', class_='mw-headline')
    years=[y.text for y in spans]

    for idx, yearr in enumerate(years):
        names_by_year[yearr] = []
        current_span = spans[idx]
        header = current_span.parent
        div = header.find_next('table')
        
        lis = div.find_all('td')
        for l in lis:
            try:
                cover_name=l.find('a').attrs['title']
                names_by_year[yearr].append(cover_name)
                names.append({
                    'name': cover_name,
                    'year': yearr
                })
#                 print(cover_name)

            except:
                pass
        
    return names

In [11]:
names3=titleB(sooup(url3)) #1990s
names3
df3=pd.DataFrame(names3)
df3.head()
#df3.count() #252

,name,year
0,Mikhail Gorbachev,1990
1,Joe Foss,1990
2,Nelson Mandela,1990
3,Michael Milken,1990
4,Soviet Union,1990


In [12]:
names4=titleB(sooup(url4)) #2000s
names4
df4=pd.DataFrame(names4)
df4.head()
#df4.count() #183 total

,name,year
0,Steve Case,2000
1,George W. Bush,2000
2,John McCain,2000
3,Leonardo DiCaprio,2000
4,George W. Bush,2000


In [19]:
# 2010-2017 (upto 316 included)
names5=titleB(sooup(url5))
names5a=names5[0:317] 
names5a
df5a=pd.DataFrame(names5)
df5a.tail()
#df5a.count() #317

,name,year
323,Greta Gerwig,2018
324,Emma González,2018
325,Viktor Orbán,2018
326,List of covers of Time magazine (2000s),References
327,List of covers of Time magazine (2020s) (page ...,References


In [36]:
#add 2018 prtion [317-328]
df5b=df5[317:328]
df5b
#df5b.to_csv('df5b.csv', index=False) 
df5c=pd.read_csv('df5b.csv', encoding='utf-8')# Then filtered, final total 8 rows
df5c

,name,year
0,Bill Gates,2018
1,Donald Trump,2018
2,Donald Trump,2018
3,Chadwick Boseman,2018
4,Black Panther,2018
5,Greta Gerwig,2018
6,Emma González,2018
7,Viktor Orbán,2018


In [40]:
#combine all the data from 1970s to 2010s
df_all=pd.concat([df1, df2,df3,df4,df5a])
df_all.head()
#df_all.count() #1157=>1449
#df_all.to_csv('names_by_years.csv', index=False) #1970-2017 # Then filtered, final total 1220

#df_allA=pd.read_csv('names_by_years.csv', encoding='utf-8') 
#df_allA.head()
#df_allA.count() 

name    1220
year    1220
dtype: int64

In [43]:
df_final=pd.concat([df_allA, df5c])
df_final.tail()
df_final.count() #1228
#df_final.to_csv('names_by_years.csv', encoding='utf-8')
final=pd.read_csv('names_by_years.csv', encoding='utf-8') 
final.count()

Unnamed: 0    1228
name          1228
year          1228
dtype: int64

### create a dataset including 'decade' column

In [45]:
# value_counts by decade (from 1970-part of 2018)
df_47yrs = pd.read_csv('names_by_years.csv', encoding='utf-8')
df_47yrs['decade']=''
df_47yrs.loc[ df_47yrs['year']>=2010,['decade']]=['2010s']
df_47yrs.loc[(df_47yrs['year']>=2000) & (df_47yrs['year']<2010) ,['decade']]=['2000s']
df_47yrs.loc[(df_47yrs['year']>=1990) & (df_47yrs['year']<2000) ,['decade']]=['1990s']
df_47yrs.loc[(df_47yrs['year']>=1980) & (df_47yrs['year']<1990) ,['decade']]=['1980s']
df_47yrs.loc[ df_47yrs['year']< 1980 ,['decade']]=['1970s']
df_47yrs.count()
df_47yrs.to_csv('decade_47yrs.csv', index=False)

# x2=df_47yrs[df_47yrs['decade']=='2010s']['name'].value_counts().to_frame(name="count") 
# x2['name']=x2.index
# x2 = x2.reset_index(drop=True)
# x2

Unnamed: 0    1228
name          1228
year          1228
decade        1228
dtype: int64

### convert csv to json

In [46]:
import sqlite3
from sqlalchemy import create_engine


In [48]:
# 'names_by_years' to sqlite
engine=create_engine('sqlite:///names_by_years_tbl.sqlite',echo=False)
df_47yrs = pd.read_csv('names_by_years.csv', encoding='utf-8')
df_47yrs.to_sql('names_by_years_tbl',con=engine)
x=engine.execute("SELECT * FROM names_by_years_tbl").fetchall()
x

[(0, 0, 'The Band', 1970),
 (1, 1, 'Najeeb Halaby', 1970),
 (2, 2, 'Biafra', 1970),
 (3, 3, 'Barry Commoner', 1970),
 (4, 4, 'Jane Fonda', 1970),
 (5, 5, 'Anita Caspary', 1970),
 (6, 6, 'Jesse Jackson', 1970),
 (7, 7, 'Günter Grass', 1970),
 (8, 8, 'Richard Nixon', 1970),
 (9, 9, 'Jim Lovell', 1970),
 (10, 10, 'Leonid Brezhnev', 1970),
 (11, 11, 'William H. Masters', 1970),
 (12, 12, 'Arthur Burns', 1970),
 (13, 13, 'Richard Nixon', 1970),
 (14, 14, 'Mike Nichols', 1970),
 (15, 15, 'Edward Heath', 1970),
 (16, 16, 'Jerry Vernon Wilson', 1970),
 (17, 17, 'Henry Ford II', 1970),
 (18, 18, 'Alaska', 1970),
 (19, 19, 'William P. Rogers', 1970),
 (20, 20, 'Howard Stein', 1970),
 (21, 21, 'Kate Millett', 1970),
 (22, 22, 'Elliott Gould', 1970),
 (23, 23, 'John Fairchild', 1970),
 (24, 24, 'Richard Nixon', 1970),
 (25, 25, 'Gamal Abdel Nasser', 1970),
 (26, 26, 'Salvador Allende', 1970),
 (27, 27, 'Adlai Stevenson III', 1970),
 (28, 28, 'Martha Beall Mitchell', 1970),
 (29, 29, 'Elmo Zumwalt'

In [49]:
#'decade_47yrs' to sqlite
engine=create_engine('sqlite:///decade_47yrs_tbl.sqlite',echo=False)
df_f= pd.read_csv('decade_47yrs.csv', encoding='utf-8')
df_f.to_sql('decade_47yrs_tbl',con=engine)
x2=engine.execute("SELECT * FROM decade_47yrs_tbl").fetchall()
x2

[(0, 0, 'The Band', 1970, '1970s'),
 (1, 1, 'Najeeb Halaby', 1970, '1970s'),
 (2, 2, 'Biafra', 1970, '1970s'),
 (3, 3, 'Barry Commoner', 1970, '1970s'),
 (4, 4, 'Jane Fonda', 1970, '1970s'),
 (5, 5, 'Anita Caspary', 1970, '1970s'),
 (6, 6, 'Jesse Jackson', 1970, '1970s'),
 (7, 7, 'Günter Grass', 1970, '1970s'),
 (8, 8, 'Richard Nixon', 1970, '1970s'),
 (9, 9, 'Jim Lovell', 1970, '1970s'),
 (10, 10, 'Leonid Brezhnev', 1970, '1970s'),
 (11, 11, 'William H. Masters', 1970, '1970s'),
 (12, 12, 'Arthur Burns', 1970, '1970s'),
 (13, 13, 'Richard Nixon', 1970, '1970s'),
 (14, 14, 'Mike Nichols', 1970, '1970s'),
 (15, 15, 'Edward Heath', 1970, '1970s'),
 (16, 16, 'Jerry Vernon Wilson', 1970, '1970s'),
 (17, 17, 'Henry Ford II', 1970, '1970s'),
 (18, 18, 'Alaska', 1970, '1970s'),
 (19, 19, 'William P. Rogers', 1970, '1970s'),
 (20, 20, 'Howard Stein', 1970, '1970s'),
 (21, 21, 'Kate Millett', 1970, '1970s'),
 (22, 22, 'Elliott Gould', 1970, '1970s'),
 (23, 23, 'John Fairchild', 1970, '1970s'),


### scraping the personal info (birth) from google site

In [80]:
names=df_f['name']
names[0:2]

0         The Band
1    Najeeb Halaby
Name: name, dtype: object

In [85]:
def info(names):
    data =[]
    for name in names:
 #       time.sleep(1)
        url = "https://www.google.com/search?q="
        finalurl = url + name
        browser.visit(finalurl)
        html = browser.html
        soup = bs(html, 'html.parser')
        try:
            birthdate = soup.find('span', class_='LrzXr').get_text()
    #       image = soup.select_one('div.IArpUc a g-img img').get("src")

            data.append({'name':name, 'birthdate': birthdate})
    #       data.append({'name':name, 'birthdate': birthdate, 'image': image})
     #      print(name, birthdate)
        except Exception as e:
            pass

    return data


In [86]:
data1=info(names[0:2])
data1

[{'name': 'The Band', 'birthdate': 'Toronto, Canada (1964)'},
 {'name': 'Najeeb Halaby', 'birthdate': 'November 19, 1915, Dallas, TX'}]

### study of the occupation from 'full_merge.csv'

In [280]:
dg=pd.read_csv('full_merge.csv', encoding="utf-8")
dg1=dg[['id','name','types']]
dg1
dgg=dg1[dg1['name'].str.contains("Jesse Jackson",na = False)]
dgg

,id,name,types
9,42512,"Rev Jesse Jackson, Sr","['Person', 'Political Candidate']"
10,29461,Jesse Jackson JR for Congress,"['Organization', 'Political Fundraising Commit..."
11,171244,Jesse Jackson Jr for Congress Committee,['Organization']
12,235612,Jesse Jackson for President '88 Cmte,"['Organization', 'Political Fundraising Commit..."
958,42512,"Rev Jesse Jackson, Sr","['Person', 'Political Candidate']"
959,29461,Jesse Jackson JR for Congress,"['Organization', 'Political Fundraising Commit..."
960,171244,Jesse Jackson Jr for Congress Committee,['Organization']
961,235612,Jesse Jackson for President '88 Cmte,"['Organization', 'Political Fundraising Commit..."
1023,42512,"Rev Jesse Jackson, Sr","['Person', 'Political Candidate']"
1024,29461,Jesse Jackson JR for Congress,"['Organization', 'Political Fundraising Commit..."


In [164]:
dg3=dg1[dg1['types'].str.contains("Person",na = False)]
dg3.head(5)
#dg3.count()  #2017 counts
#dg3['types'].unique()

,id,name,types
2,98373,Najeeb Elias Halaby,"['Person', 'Business Person', 'Public Official..."
3,218983,Najeeb Halaby,"['Person', 'Business Person']"
4,109517,Barry Commoner,"['Person', 'Academic', 'Political Candidate']"
5,90649,Jane Fonda,"['Person', 'Media Personality']"
6,150103,Anita Caspary,['Person']


In [223]:
#.str.replace('[','').str.replace(']','').str.replace("'",'').str.split( ",", expand=True)

s4=dg3['types'].str.replace("'Person'," ,'').str.replace("'Person'" ,'others')
dg4=pd.DataFrame(s4)

dg5=pd.merge(dg3,dg4,left_index=True, right_index=True)
#dg5['name'].value_counts()
#dg5.count()#2017 rows
dg5.head()


,id,name,types_x,types_y
2,98373,Najeeb Elias Halaby,"['Person', 'Business Person', 'Public Official...","[ 'Business Person', 'Public Official', 'Lawyer']"
3,218983,Najeeb Halaby,"['Person', 'Business Person']",[ 'Business Person']
4,109517,Barry Commoner,"['Person', 'Academic', 'Political Candidate']","[ 'Academic', 'Political Candidate']"
5,90649,Jane Fonda,"['Person', 'Media Personality']",[ 'Media Personality']
6,150103,Anita Caspary,['Person'],[others]


In [282]:
dfa = df_f.drop_duplicates(subset=['name'])#1228=>774 rows
dfb = dg5.drop_duplicates(subset=['name'])#2017=>474 rows

new_df = pd.merge(dfa, dfb, how='left', on='name')
new_df.head(100)
# new_df.count() #774rows
# new_dff=new_df[new_df['name'].str.contains("Hillary Clinton",na = False)]
# new_dff

,Unnamed: 0,name,year,decade,id,types_x,types_y
0,0,The Band,1970,1970s,NaN,NaN,NaN
1,1,Najeeb Halaby,1970,1970s,218983.0,"['Person', 'Business Person']",[ 'Business Person']
2,2,Biafra,1970,1970s,NaN,NaN,NaN
3,3,Barry Commoner,1970,1970s,109517.0,"['Person', 'Academic', 'Political Candidate']","[ 'Academic', 'Political Candidate']"
4,4,Jane Fonda,1970,1970s,90649.0,"['Person', 'Media Personality']",[ 'Media Personality']
5,5,Anita Caspary,1970,1970s,150103.0,['Person'],[others]
6,6,Jesse Jackson,1970,1970s,NaN,NaN,NaN
7,7,Günter Grass,1970,1970s,150097.0,"['Person', 'Media Personality']",[ 'Media Personality']
8,8,Richard Nixon,1970,1970s,56500.0,['Person'],[others]
9,9,Jim Lovell,1970,1970s,NaN,NaN,NaN


In [ ]:
#job1:' Business Person', ' Academic', ' Media Personality', None,
#       ' Elected Representative', ' Political Candidate',
#       ' Public Official', ' Lobbyist', ' Lawyer']

#job2:' Public Official', None, ' Political Candidate', ' Lawyer',
#       ' Elected Representative', ' Lobbyist', ' Business Person',
#       ' Academic', ' Media Personality'

#jjob3:' Lawyer', None, ' Public Official', ' Business Person',
#        ' Lobbyist', ' Political Candidate', ' Elected Representative',
#        ' Public Intellectual', ' Media Personality', ' Academic'
        
# job4:' Lawyer', ' Public Official', ' Public Intellectual'
# job5:' Lawyer', ' Elected Representative'
# job6: Business Person

Total categories: 'Business Person', ' Academic', ' Media Personality',
       ' Elected Representative', ' Political Candidate',
       ' Public Official', ' Lobbyist', ' Lawyer',' Public Intellectual', others
    No use

In [125]:
dg3.to_csv('dg3.csv', encoding='utf-8')

### wordCloud

In [ ]:
import numpy as np
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import matplotlib.pyplot as plt
from function_creation import draw_chart,draw_chart2, print_label, calc_means, calc_sems, count_sample

### metadata

In [ ]:
meta2=[{'decade':'overall','fact1':'Cold war ended, USSR collapsed',
        'fact2':'vietnam war ended',
        'fact3':'Apollo 15 lands on the moon',
        'fact4':'Islaeli-PLO tension ','fact5':'watergate',
        'fact6':'Los Angeles riots',
        'fact7':'Microsoft, Apple, google, etc started.',
        'fact8':'Donald J. Trump'},

       {'decade':'1970s','fact1':'Vietnam war ends',
        'fact2':'Apollo 15 lands on the moon',
        'fact3':'President Nixon makes an unprecedented visit to Communist China',
        'fact4':'U.S. signs peace pact and troops pull out of Vietnam',
        'fact5':'Nixon resigns over the Watergate Scandal',
        'fact6':'Pol Pot and the Khmer Rouge take over in Cambodia and begin a blood bath.',
        'fact7':'Microsoft, Apple coorporation founded',
        'fact8':'Jimmy Carter is elected the 39th President.'},

       {'decade':'1980s', 'fact1':'Ronald Reagan is elected president.',
        'fact2':'John Lennon of the Beatles is shot dead',
        'fact3':'The first cases of AIDS are identified',
        'fact4':'President Reagan is wounded by gunshots',
        'fact5':'Rome Pope John Paul II is wounded',
        'fact6':'Geroge Bush elected U.S. President',
        'fact7':'Soviet leader Leonid Brezhnev dies', 
        'fact8':'An explosion in the nuclear power plant at Chernobyl'},

       {'decade':'1990s', 'fact1':'The Cold War formally ends,& the USSR dissolves.',
        'fact2':'Gulf War against Iraq',
        'fact3':'Bill Clinton is elected president',
        'fact4':'Los Angeles riots after Rodney King incidence',       
        'fact5':'Rwandan genocide begins in Africa.',
        'fact6':'Timothy McVeigh bombs the Oklahoma City',
        'fact7':'outbreak of Mad Cow Disease in england', 
        'fact8':'President Bill Clinton: sex scandal,impeached'},

       {'decade':'2000s', 'fact1':'George W. Bush becomes President.',
        'fact2':'Attack on the World Trade Center:  September 11',
        'fact3':'Israeli tanks and warplanes attack West Bank towns',
        'fact4':'President Bush and Britain attacks Iraq ',
        'fact5':'Tsunami in Asia. At least 225,000 are killed.', 
        'fact6':'George W. Bush is re-elected.',
        'fact7':'Facebook takes off.', 
        'fact8':'Hurricane Katrina destroys much of the Gulf Coast'},

       {'decade':'2010s', 'fact1':'U.S. Navy SEALs Take Down Osama Bin Laden',
        'fact2':'Rise of ISIS',             
        'fact3':'Deepwater Horizon Explosion in the Gulf of Mexico',
        'fact4':'Sharing Economy Pioneers Airbnb and Uber',        
        'fact5':'U.S. Supreme Court Allows Same-Sex Marriage', 
        'fact6':'Donald J. Trump Becomes the 45th President of the United States',
        'fact7':'Facebook takes off.  WikiLeaks', 
        'fact8':'North-South Korea Tensions'}
     ]

df1_=pd.DataFrame(meta2)
df1_.to_csv('meta2.csv', index=False) 
df1_.head()  